In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/videotest1112/..          .mp4
/kaggle/input/videotest1112/Wayne Rooney says Ruben Amorim and players must take responsibility for Man Utd form (1).mp4
/kaggle/input/videotest1112/Hamas and Israel begin indirect talks to end devastating war in Gaza (1).mp4


In [3]:
# Step 1: FIXED Kaggle dependencies with proper numpy version
!pip uninstall -y numpy transformers accelerate
!pip install numpy==1.24.3
!pip install transformers==4.30.0
!pip install accelerate==0.20.3
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python moviepy librosa spacy nltk
!pip install speechrecognition pydub sumy textblob
!pip install facenet-pytorch
!pip install deep-translator
!pip install arabic-reshaper
!pip install python-bidi
!pip install pyarabic

# Download spaCy models
!python -m spacy download en_core_web_sm -q
!python -m spacy download fr_core_news_sm -q

print("✅ Kaggle dependencies installed successfully!")

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 64.4 MB/s eta 0:00:00:00:01:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.18.0 requires transformers>=4.33.1, which is not installed.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, which is not installed.
peft 0.16.0 requires accelerate>=0.21.0, which is not installed.
peft 0.16.0 requires transformers, which is

In [4]:
# Step 2: Import libraries with proper error handling
import sys
import subprocess
import importlib

def safe_import(module_name, install_name=None):
    """Safely import a module with installation fallback"""
    if install_name is None:
        install_name = module_name
    
    try:
        module = importlib.import_module(module_name)
        print(f"✅ {module_name} imported successfully")
        return module
    except ImportError:
        print(f"📦 Installing {install_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", install_name])
        return importlib.import_module(module_name)

# Import critical packages
print("🔄 Importing libraries...")
import numpy as np
print(f"✅ NumPy version: {np.__version__}")

import cv2
import torch
print(f"✅ PyTorch version: {torch.__version__}")

# Import transformers components safely
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
    print("✅ Transformers imported successfully")
except Exception as e:
    print(f"❌ Transformers import failed: {e}")
    # Try alternative import
    try:
        import transformers
        print("✅ Transformers imported via alternative method")
    except Exception as e2:
        print(f"❌ Alternative import also failed: {e2}")
        raise e2

import speech_recognition as sr
from moviepy.editor import VideoFileClip
import spacy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import librosa
import warnings
from PIL import Image
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
import tempfile
import os
import json
from deep_translator import GoogleTranslator
import arabic_reshaper
from bidi.algorithm import get_display
import IPython.display as ipd
from pathlib import Path
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

🔄 Importing libraries...
✅ NumPy version: 1.26.4
✅ PyTorch version: 2.6.0+cu124
✅ Transformers imported successfully


error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

✅ All libraries imported successfully!


In [6]:
# Step 3: ULTRA-SIMPLE Video Analyzer (No transformers pipeline dependency)
class UltraSimpleVideoAnalyzer:
    def __init__(self):
        self.setup_ultra_simple_models()
        
    def setup_ultra_simple_models(self):
        """Setup models without using transformers pipeline"""
        print("🔄 Setting up ultra-simple models...")
        
        # Use OpenCV for face detection only
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Speech recognition
        self.recognizer = sr.Recognizer()
        
        # Simple emotion mapping (we'll use text-based analysis)
        self.emotion_keywords = {
            'happy': ['happy', 'joy', 'excited', 'good', 'great', 'wonderful', 'amazing'],
            'sad': ['sad', 'unhappy', 'cry', 'bad', 'terrible', 'awful', 'depressed'],
            'angry': ['angry', 'mad', 'frustrated', 'annoyed', 'upset'],
            'neutral': ['ok', 'fine', 'normal', 'regular', 'usual']
        }
        
        # Topic keywords
        self.topic_keywords = {
            'sports': ['sport', 'game', 'player', 'team', 'match', 'goal', 'win'],
            'politics': ['government', 'president', 'minister', 'election', 'policy'],
            'entertainment': ['movie', 'music', 'show', 'celebrity', 'film', 'song'],
            'technology': ['computer', 'phone', 'internet', 'software', 'digital'],
            'culture': ['culture', 'tradition', 'heritage', 'art', 'history'],
            'war': ['war', 'battle', 'military', 'soldier', 'attack']
        }
        
        print("✅ Ultra-simple models ready!")
    
    def format_arabic_text(self, text):
        """Format Arabic text for proper display"""
        try:
            reshaped_text = arabic_reshaper.reshape(text)
            return get_display(reshaped_text)
        except:
            return text
    
    def get_video_paths(self, data_dir="/kaggle/input"):
        """Get video paths from Kaggle datasets"""
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
        video_paths = []
        
        if os.path.exists(data_dir):
            for root, dirs, files in os.walk(data_dir):
                for file in files:
                    if any(file.lower().endswith(ext) for ext in video_extensions):
                        video_paths.append(os.path.join(root, file))
        
        return video_paths
    
    def list_available_videos(self):
        """List all available videos"""
        video_paths = self.get_video_paths()
        
        if not video_paths:
            print("📁 No video files found in /kaggle/input/")
            print("💡 Please add a dataset with video files")
            return []
        
        print(f"🎬 Found {len(video_paths)} video files:")
        for i, path in enumerate(video_paths):
            file_size = os.path.getsize(path) / (1024 * 1024)
            print(f"   {i+1}. {os.path.basename(path)} ({file_size:.1f} MB)")
        
        return video_paths
    
    def extract_audio_ultra_simple(self, video_path):
        """Extract audio from video"""
        try:
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as temp_audio:
                audio_path = temp_audio.name
            
            video = VideoFileClip(video_path)
            video.audio.write_audiofile(audio_path, verbose=False, logger=None)
            return audio_path
        except Exception as e:
            print(f"❌ Error extracting audio: {e}")
            return None

    def transcribe_audio_ultra_simple(self, audio_path):
        """Transcribe audio using Google Speech Recognition"""
        try:
            with sr.AudioFile(audio_path) as source:
                audio = self.recognizer.record(source)
            
            # Try different languages
            languages = ['ar-SA', 'fr-FR', 'en-US']
            
            for lang in languages:
                try:
                    text = self.recognizer.recognize_google(audio, language=lang)
                    if len(text.strip()) > 2:
                        lang_name = {'ar-SA': 'arabic', 'fr-FR': 'french', 'en-US': 'english'}[lang]
                        return text, lang_name
                except:
                    continue
            
            # Final attempt without language
            try:
                text = self.recognizer.recognize_google(audio)
                return text, "auto-detected"
            except:
                return "No speech detected", "unknown"
                
        except Exception as e:
            print(f"❌ Transcription error: {e}")
            return "", "unknown"

    def detect_language_simple(self, text):
        """Simple language detection based on character analysis"""
        if len(text.strip()) < 3:
            return "unknown"
        
        # Check for Arabic characters
        arabic_chars = set('ابتثجحخدذرزسشصضطظعغفقكلمنهوي')
        if any(char in arabic_chars for char in text):
            return "arabic"
        
        # Check for French accents
        french_chars = set('éèêëàâäîïôöùûüç')
        if any(char in french_chars for char in text.lower()):
            return "french"
        
        # Default to English
        return "english"

    def translate_text_simple(self, text, source_lang):
        """Simple translation using deep-translator"""
        if source_lang == 'english':
            return text
        
        if len(text.strip()) < 2:
            return text
            
        try:
            lang_map = {'arabic': 'ar', 'french': 'fr'}
            source_code = lang_map.get(source_lang, 'auto')
            translated = GoogleTranslator(source=source_code, target='en').translate(text[:500])
            return translated
        except Exception as e:
            print(f"⚠️  Translation failed: {e}")
            return text

    def analyze_emotions_simple(self, text):
        """Simple emotion analysis based on keywords"""
        text_lower = text.lower()
        
        emotion_scores = {}
        for emotion, keywords in self.emotion_keywords.items():
            score = sum(1 for keyword in keywords if keyword in text_lower)
            emotion_scores[emotion] = score
        
        if any(score > 0 for score in emotion_scores.values()):
            dominant_emotion = max(emotion_scores, key=emotion_scores.get)
            return dominant_emotion
        else:
            return "neutral"

    def detect_faces_simple(self, video_path):
        """Simple face detection (presence/absence)"""
        cap = cv2.VideoCapture(video_path)
        face_detected = False
        
        # Check first 5 frames
        for _ in range(5):
            ret, frame = cap.read()
            if ret:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
                if len(faces) > 0:
                    face_detected = True
                    break
        
        cap.release()
        return "faces_detected" if face_detected else "no_faces"

    def classify_topic_simple(self, text):
        """Simple topic classification based on keywords"""
        text_lower = text.lower()
        
        topic_scores = {}
        for topic, keywords in self.topic_keywords.items():
            score = sum(1 for keyword in keywords if keyword in text_lower)
            topic_scores[topic] = score
        
        if any(score > 0 for score in topic_scores.values()):
            dominant_topic = max(topic_scores, key=topic_scores.get)
            confidence = topic_scores[dominant_topic] / len(text.split()) * 10
            confidence = min(confidence, 1.0)  # Cap at 1.0
            return dominant_topic, confidence
        else:
            return "general", 0.3

    def summarize_text_simple(self, text):
        """Simple text summarization"""
        if len(text) < 30:
            return "Insufficient text for summary"
        
        sentences = text.split('.')
        if len(sentences) > 2:
            # Take first and last sentence for summary
            summary = sentences[0] + '. ' + sentences[-1] + '.'
            return summary
        else:
            return text[:200] + "..."

    def analyze_video_ultra_simple(self, video_path=None):
        """Ultra-simple main analysis function"""
        if video_path is None:
            video_paths = self.get_video_paths()
            if not video_paths:
                return {"error": "No video files found in /kaggle/input/"}
            video_path = video_paths[0]
        
        print(f"🎬 Analyzing: {os.path.basename(video_path)}")
        
        # Get basic video info
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0
        cap.release()
        
        print(f"📊 Video Info: {duration:.1f}s, {total_frames} frames")
        
        # Step 1: Extract audio
        print("🔊 Extracting audio...")
        audio_path = self.extract_audio_ultra_simple(video_path)
        if not audio_path:
            return {"error": "Failed to extract audio"}
        
        # Step 2: Transcribe audio
        print("🗣️ Transcribing audio...")
        transcribed_text, detected_language = self.transcribe_audio_ultra_simple(audio_path)
        
        print(f"📝 Transcribed text length: {len(transcribed_text)} characters")
        print(f"🌐 Detected language: {detected_language}")
        
        # Step 3: Language detection
        final_language = self.detect_language_simple(transcribed_text)
        print(f"🔍 Confirmed language: {final_language}")
        
        # Step 4: Translate if needed
        if final_language != 'english':
            english_text = self.translate_text_simple(transcribed_text, final_language)
        else:
            english_text = transcribed_text
        
        # Step 5: Analyze emotions
        print("😊 Analyzing emotions...")
        text_emotion = self.analyze_emotions_simple(english_text)
        face_status = self.detect_faces_simple(video_path)
        
        # Step 6: Classify topic
        print("🏷️ Classifying topic...")
        main_topic, topic_confidence = self.classify_topic_simple(english_text)
        
        # Step 7: Generate summary
        print("📄 Generating summary...")
        video_summary = self.summarize_text_simple(english_text)
        
        # Clean up
        try:
            if os.path.exists(audio_path):
                os.unlink(audio_path)
        except:
            pass
        
        # Compile results
        results = {
            "video_info": {
                "file_name": os.path.basename(video_path),
                "duration_seconds": duration,
                "total_frames": total_frames,
                "detected_language": final_language,
                "transcribed_text_length": len(transcribed_text)
            },
            "analysis": {
                "face_detection": face_status,
                "text_emotion": text_emotion,
                "main_topic": main_topic,
                "topic_confidence": f"{topic_confidence:.1%}",
                "video_summary": video_summary
            },
            "content": {
                "original_transcription": transcribed_text[:350] + "..." if len(transcribed_text) > 350 else transcribed_text,
                "english_translation": english_text[:350] + "..." if len(english_text) > 350 else english_text
            }
        }
        
        return results
    
    def display_ultra_simple_results(self, results):
        """Display results in ultra-simple format"""
        if "error" in results:
            print(f"❌ Error: {results['error']}")
            return
        
        print("\n" + "="*60)
        print("🎯 ULTRA-SIMPLE VIDEO ANALYSIS RESULTS")
        print("="*60)
        
        print(f"\n📁 VIDEO INFORMATION:")
        print(f"   File: {results['video_info']['file_name']}")
        print(f"   Duration: {results['video_info']['duration_seconds']:.1f} seconds")
        print(f"   Language: {results['video_info']['detected_language']}")
        print(f"   Transcription Length: {results['video_info']['transcribed_text_length']} chars")
        
        print(f"\n📊 ANALYSIS RESULTS:")
        print(f"   Face Detection: {results['analysis']['face_detection']}")
        print(f"   Emotion: {results['analysis']['text_emotion']}")
        print(f"   Main Topic: {results['analysis']['main_topic']}")
        print(f"   Confidence: {results['analysis']['topic_confidence']}")
        
        print(f"\n📄 VIDEO SUMMARY:")
        if results['video_info']['detected_language'] == 'arabic':
            formatted_summary = self.format_arabic_text(results['analysis']['video_summary'])
            print(f"   {formatted_summary}")
        else:
            print(f"   {results['analysis']['video_summary']}")
        
        print(f"\n🗣️ CONTENT PREVIEW:")
        original_text = results['content']['original_transcription']
        if results['video_info']['detected_language'] == 'arabic':
            formatted_original = self.format_arabic_text(original_text)
            print(f"   Original: {formatted_original}")
        else:
            print(f"   Original: {original_text}")
        
        if results['content']['english_translation'] != original_text:
            print(f"   English: {results['content']['english_translation']}")

# Step 4: Main execution function
def run_ultra_simple_analysis():
    """Run ultra-simple analysis in Kaggle"""
    print("🚀 Starting Ultra-Simple Video Analysis")
    print("="*50)
    print("🌍 Supports: Arabic 🇸🇦 | French 🇫🇷 | English 🇺🇸")
    print("="*50)
    
    try:
        analyzer = UltraSimpleVideoAnalyzer()
        
        # List available videos
        video_paths = analyzer.list_available_videos()
        if not video_paths:
            return
        
        # Analyze first video
        results = analyzer.analyze_video_ultra_simple()
        
        # Display results
        analyzer.display_ultra_simple_results(results)
        
        # Save results
        try:
            output_file = "ultra_simple_video_analysis.json"
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\n💾 Results saved to: {output_file}")
        except Exception as e:
            print(f"⚠️  Could not save results: {e}")
            
        return results
        
    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        print("💡 This is a simplified version that should work in Kaggle")

# Run the analysis
if __name__ == "__main__":
    results = run_ultra_simple_analysis()

🚀 Starting Ultra-Simple Video Analysis
🌍 Supports: Arabic 🇸🇦 | French 🇫🇷 | English 🇺🇸
🔄 Setting up ultra-simple models...
✅ Ultra-simple models ready!
🎬 Found 3 video files:
   1. ..          .mp4 (11.9 MB)
   2. Wayne Rooney says Ruben Amorim and players must take responsibility for Man Utd form (1).mp4 (5.2 MB)
   3. Hamas and Israel begin indirect talks to end devastating war in Gaza (1).mp4 (19.0 MB)
🎬 Analyzing: ..          .mp4
📊 Video Info: 171.3s, 4283 frames
🔊 Extracting audio...
🗣️ Transcribing audio...
📝 Transcribed text length: 219 characters
🌐 Detected language: arabic
🔍 Confirmed language: arabic
😊 Analyzing emotions...
🏷️ Classifying topic...
📄 Generating summary...

🎯 ULTRA-SIMPLE VIDEO ANALYSIS RESULTS

📁 VIDEO INFORMATION:
   File: ..          .mp4
   Duration: 171.3 seconds
   Language: arabic
   Transcription Length: 219 chars

📊 ANALYSIS RESULTS:
   Face Detection: faces_detected
   Emotion: neutral
   Main Topic: war
   Confidence: 18.2%

📄 VIDEO SUMMARY:
   In th